ПАКЕТ SURPRISE

используйте данные MovieLens 1M
можно использовать любые модели из пакета
получите RMSE на тестовом сете 0.87 и ниже

In [36]:
from surprise import KNNWithMeans, KNNBasic, SVD, KNNWithZScore
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

import pandas as pd

Волшебная библиотека для рекомендашек
http://surpriselib.com/

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [7]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)

In [8]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [9]:
dataset.head()

,uid,iid,rating
0,1.0,Toy Story (1995),4.0
1,5.0,Toy Story (1995),4.0
2,7.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),2.5
4,17.0,Toy Story (1995),4.5


In [10]:
ratings.rating.min()

0.5

In [11]:
ratings.rating.max()

5.0

In [12]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [13]:
trainset, testset = train_test_split(data, test_size=.15)

## 1. KNNWithMeans

In [19]:
algo = KNNWithMeans(k=50, sim_options={'name': 'pearson_baseline', 'user_based': False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [20]:
test_pred = algo.test(testset)

In [21]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8670


0.8670486795358704

Бинго, RMSE меньше 0.87 на тесте

## 2. SVD

In [29]:
#  попробуем SVD
algo = SVD()
algo.fit(trainset)

In [27]:
test_pred = algo.test(testset)

In [30]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8689


0.8688718831177004

SVD норм, RMSE ниже 0.87

## 3. KNNBasic

In [31]:
# попробуем простой KNN
algo = KNNBasic()
algo.fit(trainset)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [32]:
test_pred = algo.test(testset)

In [33]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.9362


0.9362212295419103

KNNBasic заметно хуже KNNWithWeans и SVD, RMSE ниже 0.93

## 4. KNNWithZScore

In [52]:
algo = KNNWithZScore(k=50, sim_options={'name': 'pearson_baseline'})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


In [53]:
test_pred = algo.test(testset)

In [54]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8811


0.8811359087020408

Оскар уходит KNNWithMeans и SVD